# Group Exercise CLV - Group 6
## Churn Modeling

In [ ]:
# turns off warnings in the jupyter notebook
import warnings
warnings.filterwarnings('ignore')

## Importing some standard packages, as well as packages needed for modeling:

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier)
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics

from sklearn.feature_selection import SelectFromModel

import statsmodels.api as sm

import re
import pickle
from statsmodels.stats.outliers_influence import variance_inflation_factor    


## import modeling data created in the Data Prep pipeline. Change the path in below cell

In [ ]:
import pickle
with open('combined_df.pkl', 'rb') as file:
    combined_df = pickle.load(file)

## Doing some basic exploratory analysis:

In [ ]:
""" Analyzing average values of numeric variables, by the levels of the binary target variable """
print(" ") 
print("averages by churn ind")
print(" ") 
print(combined_df.groupby('churn_year_plus1_ind')['log_purch_amt_life'].mean())

print(" ") 
print(" ") 
print(combined_df.groupby('churn_year_plus1_ind')['scheduled_purchase_flg'].mean())

print(" ") 
print(" ") 
print(combined_df.groupby('churn_year_plus1_ind')['months_since_last_trans'].mean())

In [ ]:
""" A boxplot to investigate the distribution of numeric variables by the levels of the target """

combined_df.boxplot(column='log_order_cnt_36mo', by='churn_year_plus1_ind',figsize=(15,15))

## Printing variable stats:

In [ ]:
print(" summary stats on combined data ") 
print(combined_df.describe(include='all').transpose())

## Dropping variables we don't want included in modeling, and creating new version of data for modeling:

In [ ]:
""" Any categorical variable for which we did not create dummy indicators        """
"""    and other variables we don't want in the model, or which don't make sense """
"""    to be included in model training, we are dropping here:                   """

""" Note:  we also want to drop acct_ID, but we are going to do that after partioning """
"""        because we need acct_ID for partioned sets to for reference later          """

x_train_df = combined_df.drop(['Zip','DMA','Area_Cd','activation_date','event_date',
                            'ST','tier','tier_prev','demog_ownrent','Region'
                              ],axis=1)

x_train_df.shape

In [ ]:
# Changing any boolean columns into int datatypes to prevent errors
boolean_columns = list(x_train_df.dtypes[x_train_df.dtypes == 'bool'].index)
x_train_df[boolean_columns] = x_train_df[boolean_columns].astype('int')

## Partitioning data into Training and Test sets, creating X vs Y sets, and
##     creating dataframes with just account IDs to be used to rejoin data later:

In [ ]:
x_train_df, x_test_df = train_test_split(x_train_df, test_size = 0.30)

""" will need the acct_IDs to rejoin data later, so pulling out now before dropping from actual model training data """

train_acct_IDs_df = x_train_df[['acct_ID']]
test_acct_IDs_df  = x_test_df[['acct_ID']]

""" the target (or dependent) variable is stored in it's own dataframe """

y_train_df = x_train_df[['churn_year_plus1_ind']]
y_test_df  = x_test_df[['churn_year_plus1_ind']]

""" and we want to drop acct_ID and the target variable from the dataframe of independent variables """
""" PLUS we want to drop the target (future timeframe) variables for margin modeling                """

x_train_df = x_train_df.drop(['acct_ID','churn_year_plus1_ind', 'log_margin_Y_pos_1', 'margin_Y_pos_1'
                             ],axis=1)

x_test_df = x_test_df.drop(['acct_ID','churn_year_plus1_ind', 'log_margin_Y_pos_1', 'margin_Y_pos_1'
                             ],axis=1)

x_train_df.shape

## Starting with a scikit_learn Logistic Regression Model
##    and using that modeling algorithm to select variables.

## Note:  change the C parameter and rerun to get reasonable number of selected vars.

In [ ]:
""" models penalized with the L1 norm have sparse solutions: many of their estimated coefficients are zero """
""" SelectFromModel can be used to to select effects with non-zero coefficients """
""" the parameter C controls the sparsity: the smaller C the fewer features selected """

""" using the ravel simply to avoid warning in log """
y_train_array=np.ravel(y_train_df)

""" start with C = 1.0, then try 0.1, etc """
model1 = linear_model.LogisticRegression(C=0.01, solver='liblinear', penalty='l1', tol=0.01)
model1_fit = model1.fit(x_train_df, y_train_array)
model1_selects = SelectFromModel(model1_fit, prefit=True)
"""x_train_selected_df = model1_selects.transform(x_train_df)"""

""" in lieu of commented-out transform method above, can use this method to explicitly define columns to keep """
x_train_selected_df = x_train_df[x_train_df.columns[model1_selects.get_support()]]
x_test_selected_df  = x_test_df[x_test_df.columns[model1_selects.get_support()]]

coefficients = pd.DataFrame(model1_fit.coef_)
coefficients = np.transpose(coefficients[coefficients.columns[model1_selects.get_support()]])
coefficients = coefficients.reset_index(drop=True)

coefficients_match = pd.concat([pd.DataFrame(x_train_selected_df.columns),coefficients], axis = 1)

coefficients.shape 

In [ ]:
print(" ") 
print("Coefficients for selected variables")
coefficients_match.head(40)

## Need to introduce variable reduction that addresses multicolinearity:

In [ ]:
# function to calculate VIF for all independent variables and drop variables with VIF > 10

def calculate_vif_(X, thresh=10):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
        maxloc = vif.index(max(vif))
    # Drop variables with VIF values greater than 10    
    if max(vif) > thresh: 
        print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
        variables = np.delete(variables, maxloc)
        dropped=True
        
        print('Remaining variables:')
        print(X.columns[variables])
    return X.iloc[:, variables]

x_train_selected_df=calculate_vif_(x_train_selected_df, 10.0)
        
x_train_selected_df.shape

In [ ]:
# function to calculate VIF for all independent variables and drop variables with VIF > 10
thresh=10

X = x_train_selected_df

dropped=True

while dropped==True:
    cols = X.columns
    variables = np.arange(X.shape[1])
    
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
        maxloc = vif.index(max(vif))

    # Drop variables with VIF values greater than 10    
    if max(vif) > thresh: 
        print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
        variables = np.delete(variables, maxloc)
        dropped=True
        
        print('Remaining Shape:')
        print(X.shape)
    else:
        dropped=False
    X = X.iloc[:, variables]

x_train_selected_df = X

## Run the step above, repeatedly, until no more variables are dropped.

In [ ]:
""" reducing the test set to have the same columns as the training set """

x_test_selected_df=x_test_selected_df[x_train_selected_df.columns]
print('Remaining variables in test set:')
print(x_test_selected_df.columns)
        
x_test_selected_df.shape

## Using the statsmodels package to see "friendlier" looking Logistic output:

In [ ]:
""" for the statsmodels, need to add a constant to fit intercept """

x_train_selected_df = sm.add_constant(x_train_selected_df, has_constant='add')
x_test_selected_df  = sm.add_constant(x_test_selected_df, has_constant='add')

""" modeling with selected variables from steps above """

model = sm.Logit(y_train_df, x_train_selected_df)
result = model.fit(maxiter=100)

print(" ") 
print("Logistic model with selected variables")
print(" ") 
print(result.summary2())
print('AIC: ', result.aic)
print('BIC: ', result.bic)

## Using the statsmodels Logit model to score the data and evaluate goodness-of-fit:

In [ ]:
""" predicted values using the model """
""" using numpy copy to store result as an array """
""" manually adding a column header """
predict_train = pd.DataFrame(np.copy(result.predict(x_train_selected_df)), columns=["P_churn_year_plus1_ind"])
predict_test = pd.DataFrame(np.copy(result.predict(x_test_selected_df)), columns=["P_churn_year_plus1_ind"])

print ('AROC train: ', metrics.roc_auc_score(y_train_df, predict_train))
print ('AROC test: ', metrics.roc_auc_score(y_test_df, predict_test))

""" note that the index for the predicted values vector """
""" is reset, and thus complicates a join back with the original """
""" thus, reset the index on the original data, then join with predicted """
""" WARNING: you have to do the same, to join back with dataframe of regressors """
y_train_df = y_train_df.reset_index(drop=True)
y_test_df = y_test_df.reset_index(drop=True)

""" joining predicted and actuals """
""" NOTE:  for reference, for now...to be used more later to create complete """
"""     dataframe with all obs and inputs and outputs """
train_accounts = train_acct_IDs_df.reset_index(drop=True)
pred_v_actual_train = y_train_df.join(predict_train)
pred_v_actual_train = pred_v_actual_train.join(train_accounts)

test_accounts = test_acct_IDs_df.reset_index(drop=True)
pred_v_actual_test = y_test_df.join(predict_test)
pred_v_actual_test = pred_v_actual_test.join(test_accounts)

print(" ") 
print("MSE train:", mean_squared_error(y_train_df, predict_train))
print("MSE test:", mean_squared_error(y_test_df, predict_test))

print(" ") 
print("testing that the joining process works. if so, these MSE should match the ones above.")
print("mean squared error train:" , (np.mean((pred_v_actual_train['churn_year_plus1_ind']-pred_v_actual_train['P_churn_year_plus1_ind'])**2)))
print("mean squared error testn:" , (np.mean((pred_v_actual_test['churn_year_plus1_ind']-pred_v_actual_test['P_churn_year_plus1_ind'])**2)))

pred_v_actual_test.boxplot(column='P_churn_year_plus1_ind', by='churn_year_plus1_ind',figsize=(15,15))

## Now to try a more advanced Machine Learning algorithm to see if we can get a better model:
**NOTE** - This cell might take some time to run depending on number of estimators 

Make function for scoring model

Include optional *fullstats* parameter which determines if full stats and plots are generated

In [ ]:
def score_model(my_model, y_test_df, y_train_df, fullstats=True):
    """ returns overall accuracy"""
    accuracy = my_model.score(x_test_selected_df, y_test_df)
    print("GB overall accuracy:", accuracy)

    """ predicted values using the model """
    """ using numpy copy to store result as an array """
    """ manually adding a column header """
    predict_train = pd.DataFrame(np.copy(my_model.predict_proba(x_train_selected_df)[:, 1]), columns=["P_churn_year_plus1_ind"])
    predict_test = pd.DataFrame(np.copy(my_model.predict_proba(x_test_selected_df)[:, 1]), columns=["P_churn_year_plus1_ind"])

    print ('AROC train: ', metrics.roc_auc_score(y_train_df, predict_train))
    print ('AROC test: ', metrics.roc_auc_score(y_test_df, predict_test))

    if fullstats:

        """ note that the index for the predicted values vector """
        """ is reset, and thus complicates a join back with the original """
        """ thus, reset the index on the original data, then join with predicted """
        """ WARNING: you have to do the same, to join back with dataframe of regressors """
        y_train_df = y_train_df.reset_index(drop=True)
        y_test_df = y_test_df.reset_index(drop=True)

        train_accounts = train_acct_IDs_df.reset_index(drop=True)
        pred_v_actual_train = y_train_df.join(predict_train)
        pred_v_actual_train = pred_v_actual_train.join(train_accounts)


        test_accounts = test_acct_IDs_df.reset_index(drop=True)
        pred_v_actual_test = y_test_df.join(predict_test)
        pred_v_actual_test = pred_v_actual_test.join(test_accounts)

        print(" ") 
        print("MSE train:", mean_squared_error(y_train_df, predict_train))
        print("MSE test:", mean_squared_error(y_test_df, predict_test))

        print(" ") 
        print("testing that the joining process works. if so, these MSE should match the ones above.")
        print("mean squared error train:" , (np.mean((pred_v_actual_train['churn_year_plus1_ind']-pred_v_actual_train['P_churn_year_plus1_ind'])**2)))
        print("mean squared error test:" , (np.mean((pred_v_actual_test['churn_year_plus1_ind']-pred_v_actual_test['P_churn_year_plus1_ind'])**2)))


        pred_v_actual_test.boxplot(column='P_churn_year_plus1_ind', by='churn_year_plus1_ind',figsize=(15,15))

        """ plotting AROC with each iteration of the Gradient Boosting algorithm """

        """ converting Y dataframes into arrays as needed for logic below """
        y_traint_array = y_train_df.values
        y_test_array = y_test_df.values

        test_AROC  = np.zeros((params['n_estimators'],), dtype=np.float64)
        train_AROC = np.zeros((params['n_estimators'],), dtype=np.float64)

        for i, y_pred in enumerate(my_model.staged_predict_proba(x_test_selected_df)):
            test_AROC[i] = metrics.roc_auc_score(y_test_array, y_pred[:, 1])

        for i, y_pred in enumerate(my_model.staged_predict_proba(x_train_selected_df)):
            train_AROC[i] = metrics.roc_auc_score(y_train_array, y_pred[:, 1])

        plt.figure(figsize=(15,15))
        plt.subplot(1, 2, 1)
        plt.title('AROC by iteration')
        plt.plot(np.arange(params['n_estimators']) + 1, train_AROC, 'b-',
                label='Training Set AROC')
        plt.plot(np.arange(params['n_estimators']) + 1, test_AROC, 'r-',
                label='Test Set AROC')
        plt.legend(loc='upper right')
        plt.xlabel('Boosting Iterations')
        plt.ylabel('AROC')

Gradient Boosting Model 1 (model2)

In [ ]:
""" Gradient Boosting """
params = {'n_estimators': 50, 'min_samples_split': 2,  'max_depth': 5, 'learning_rate': 0.01}
model2 = GradientBoostingClassifier(**params)

""" had to switch the order of X and Y for the fit to work with GradientBoosting """
result2 = model2.fit(x_train_selected_df, y_train_array)

score_model(result2, y_test_df, y_train_df, fullstats=False)

Gradient Boosting Model 2 (model3)

In [ ]:
""" Gradient Boosting """
params = {'n_estimators': 250, 'min_samples_split': 2,  'max_depth': 5, 'learning_rate': 0.01}
model3 = GradientBoostingClassifier(**params)

""" had to switch the order of X and Y for the fit to work with GradientBoosting """
result3 = model3.fit(x_train_selected_df, y_train_array)

score_model(result3, y_test_df, y_train_df, fullstats=False)

Gradient Boosting Model 3 (model4)

In [ ]:
""" Gradient Boosting """
params = {'n_estimators': 50, 'min_samples_split': 2,  'max_depth': 5, 'learning_rate': 0.1}
model4 = GradientBoostingClassifier(**params)

""" had to switch the order of X and Y for the fit to work with GradientBoosting """
result4 = model4.fit(x_train_selected_df, y_train_array)

score_model(result4, y_test_df, y_train_df, fullstats=False)

### Champion Churn Model
The gradient boosting model with 0.01 learning rate and n_estimators=50 will be chosen as champion because it had best performance while also being fastest in compute time

In [ ]:
score_model(result2, y_test_df, y_train_df, fullstats=True)

## Feature Importance for the Gradient Boosting Model:

In [ ]:
feature_importance = model2.feature_importances_
""" make importances relative to max importance """
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(50,50))
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, x_train_selected_df.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

#### Append the train and test sets, and keeping the acct_IDs and predicted churn probabilities

In [ ]:
""" append together train and test predicted values, keeping only acct ID and predicted values """

pred_v_actual_train = pd.concat([pred_v_actual_train, pred_v_actual_test], ignore_index=True)

pred_v_actual_train = pred_v_actual_train.drop(['churn_year_plus1_ind'],axis=1)

""" and then save predicted values by acct ID to csv """
pred_v_actual_train.to_csv('churn_scores.csv',index=False)